**Question 2 -**
Implement 5 different CNN architectures with a comparison table for CIFAR 10
dataset using the PyTorch library

**Note -**
1. The model parameters for each architecture should not be more than 10000 parameters.

2. Code comments should be given for proper code understanding

In [ ]:
#import libraries
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchsummary import summary

In [ ]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# Load and normalize CIFAR10
transform = transforms.Compose(
    [transforms.ToTensor(),# Convert PIL image to PyTorch tensor
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]) # Normalize the image pixels

In [ ]:
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True,download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False,download=True, transform=transform)


In [ ]:
# Create data loaders
batch_size = 128
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Check the size of the loaded data
print("Training data size:", len(train_dataset))
print("Testing data size:", len(test_dataset))

In [ ]:
classes = ('plane', 'car', 'bird', 'cat','deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# functions to show an image
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# get some random training images
dataiter = iter(train_loader)
images, labels = next(dataiter)

# show images
imshow(torchvision.utils.make_grid(images))
# print labels
print(' '.join(f'{classes[labels[j]]:5s}' for j in range(batch_size)))

In [ ]:
# Hyperparameters
num_epochs = 30
batch_size = 128
learning_rate = 0.001

## Model 1

In [ ]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()

        self.conv1 = nn.Conv2d(3, 6, kernel_size=5)  # Modified input channels to 3
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=2)

        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=2)

        self.flatten = nn.Flatten()
        self.fc = nn.Linear(16 * 5 * 5, 10)  # Adjusted input size to 16*5*5
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)

        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)

        x = self.flatten(x)
        x = self.fc(x)
        x = self.softmax(x)
        return x

In [ ]:
# Create the LeNet-5 model
model1 = Model().to(device)

In [ ]:
# Print the model summary
summary(model1, (3, 32, 32))  # Assuming RGB images of size 32x32

In [ ]:
# Loss and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model1.parameters(), lr=learning_rate)

In [ ]:
#Model Training
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # Move inputs and labels to the same device as the model
        inputs = inputs.to(device)
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model1(inputs)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

In [ ]:
# Evaluation
model1.eval()  # Set the model to evaluation mode

correct = 0
total = 0

with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model1(images)
        _, predicted = torch.max(outputs.data, 1)
        
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

acc_model1 = 100 * correct / total
print(f'Test Accuracy: {acc_model1:.2f}%')


## Model 2

In [ ]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()

        self.conv1 = nn.Conv2d(3, 8, kernel_size=3)
        self.relu1 = nn.ReLU()
        self.batchnorm1 = nn.BatchNorm2d(8, track_running_stats=True)
        self.dropout1 = nn.Dropout2d(0.1)

        self.conv2 = nn.Conv2d(8, 16, kernel_size=3)
        self.relu2 = nn.ReLU()
        self.batchnorm2 = nn.BatchNorm2d(16, track_running_stats=True)
        self.dropout2 = nn.Dropout2d(0.1)

        self.conv3 = nn.Conv2d(16, 8, kernel_size=1)
        self.relu3 = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=2)

        self.conv4 = nn.Conv2d(8, 10, kernel_size=3)
        self.relu4 = nn.ReLU()
        self.batchnorm3 = nn.BatchNorm2d(10, track_running_stats=True)
        self.dropout3 = nn.Dropout2d(0.1)

        self.conv5 = nn.Conv2d(10, 10, kernel_size=3)
        self.relu5 = nn.ReLU()
        self.batchnorm4 = nn.BatchNorm2d(10, track_running_stats=True)
        self.dropout4 = nn.Dropout2d(0.1)

        self.conv6 = nn.Conv2d(10, 10, kernel_size=3)
        self.relu6 = nn.ReLU()
        self.batchnorm5 = nn.BatchNorm2d(10, track_running_stats=True)
        self.dropout5 = nn.Dropout2d(0.1)

        self.conv7 = nn.Conv2d(10, 10, kernel_size=3)
        self.relu7 = nn.ReLU()
        self.batchnorm6 = nn.BatchNorm2d(10, track_running_stats=True)
        self.dropout6 = nn.Dropout2d(0.1)

        self.conv8 = nn.Conv2d(10, 10, kernel_size=4)

        self.flatten = nn.Flatten()

        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.batchnorm1(x)
        x = self.dropout1(x)

        x = self.conv2(x)
        x = self.relu2(x)
        x = self.batchnorm2(x)
        x = self.dropout2(x)

        x = self.conv3(x)
        x = self.relu3(x)
        x = self.maxpool(x)

        x = self.conv4(x)
        x = self.relu4(x)
        x = self.batchnorm3(x)
        x = self.dropout3(x)

        x = self.conv5(x)
        x = self.relu5(x)
        x = self.batchnorm4(x)
        x = self.dropout4(x)

        x = self.conv6(x)
        x = self.relu6(x)
        x = self.batchnorm5(x)
        x = self.dropout5(x)

        x = self.conv7(x)
        x = self.relu7(x)
        x = self.batchnorm6(x)
        x = self.dropout6(x)

        x = self.conv8(x)

        x = self.flatten(x)

        x = self.softmax(x)
        return x

In [ ]:
# Create the model
model2 = Model().to(device)

In [ ]:
# Print the model summary
summary(model2, (3, 32, 32))  # Assuming RGB images of size 32x32

In [ ]:
# Loss and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model2.parameters(), lr=learning_rate)

In [ ]:
#Model Training
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # Move inputs and labels to the same device as the model
        inputs = inputs.to(device)
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model2(inputs)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

In [ ]:
# Evaluation
model2.eval()  # Set the model to evaluation mode

correct = 0
total = 0

with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model2(images)
        _, predicted = torch.max(outputs.data, 1)
        
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

acc_model2 = 100 * correct / total
print(f'Test Accuracy: {acc_model2:.2f}%')

## Model 3

In [ ]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()

        self.conv1 = nn.Conv2d(3, 8, kernel_size=3) # Channel dimension = 30*30*10 , receptive field 3*3
        self.relu1 = nn.ReLU()
        self.batchnorm1 = nn.BatchNorm2d(8,track_running_stats=True)
        self.dropout1 = nn.Dropout2d(0.1)

        self.conv2 = nn.Conv2d(8, 16, kernel_size=3) # Channel dimension = 24*24*10 , receptive field 5*5
        self.relu2 = nn.ReLU()
        self.batchnorm2 = nn.BatchNorm2d(16,track_running_stats=True)
        self.dropout2 = nn.Dropout2d(0.1)

        self.conv3 = nn.Conv2d(16, 8, kernel_size=1) # Channel dimension = 24*24*10 , receptive field 5*5
        self.relu3 = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=2) # Channel dimension = 12*12*10 , receptive field 10*10

        self.conv4 = nn.Conv2d(8, 8, kernel_size=3) # Channel dimension = 10*10*10 , receptive field 12*12
        self.relu4 = nn.ReLU()
        self.batchnorm3 = nn.BatchNorm2d(8,track_running_stats=True)
        self.dropout3 = nn.Dropout2d(0.1)

        self.conv5 = nn.Conv2d(8, 8, kernel_size=3) # Channel dimension = 8*8*10 , receptive field 14*14
        self.relu5 = nn.ReLU()
        self.batchnorm4 = nn.BatchNorm2d(8,track_running_stats=True)
        self.dropout4 = nn.Dropout2d(0.1)

        self.conv6 = nn.Conv2d(8, 10, kernel_size=3) # Channel dimension = 6*6*10 , receptive field 16*16
        self.relu6 = nn.ReLU()
        self.batchnorm5 = nn.BatchNorm2d(10,track_running_stats=True)
        self.dropout5 = nn.Dropout2d(0.1)

        self.conv7 = nn.Conv2d(10, 10, kernel_size=3) # Channel dimension = 4*4*10 , receptive field 18*18
        self.relu7 = nn.ReLU()
        self.batchnorm6 = nn.BatchNorm2d(10,track_running_stats=True)
        self.dropout6 = nn.Dropout2d(0.1)

        #self.conv8 = nn.Conv2d(10, 10, kernel_size=4) # Channel dimension = 24*24*10 , receptive field 5*5

        self.flatten = nn.Flatten()

        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.batchnorm1(x)
        x = self.dropout1(x)

        x = self.conv2(x)
        x = self.relu2(x)
        x = self.batchnorm2(x)
        x = self.dropout2(x)

        x = self.conv3(x)
        x = self.relu3(x)
        x = self.maxpool(x)

        x = self.conv4(x)
        x = self.relu4(x)
        x = self.batchnorm3(x)
        x = self.dropout3(x)

        x = self.conv5(x)
        x = self.relu5(x)
        x = self.batchnorm4(x)
        x = self.dropout4(x)

        x = self.conv6(x)
        x = self.relu6(x)
        x = self.batchnorm5(x)
        x = self.dropout5(x)

        x = self.conv7(x)
        x = self.relu7(x)
        x = self.batchnorm6(x)
        x = self.dropout6(x)

        #x = self.conv8(x)

        x = self.flatten(x)

        x = self.softmax(x)
        return x

In [ ]:
# Create the model
model3 = Model().to(device)

In [ ]:
# Print the model summary
summary(model3, (3, 32, 32))  # Assuming RGB images of size 32x32

In [ ]:
# Loss and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model3.parameters(), lr=learning_rate)

In [ ]:
#Model Training
for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # Move inputs and labels to the same device as the model
        inputs = inputs.to(device)
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model3(inputs)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

In [ ]:
# Evaluation
model3.eval()  # Set the model to evaluation mode

correct = 0
total = 0

with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model3(images)
        _, predicted = torch.max(outputs.data, 1)
        
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

acc_model3 = 100 * correct / total
print(f'Test Accuracy: {acc_model3:.2f}%')

In [ ]:
import pandas as pd

# Create a DataFrame
data = {
    'Architecture': ['Model1', 'Model2','Model3'],
    'Test Accuracy': [acc_model1,acc_model2,acc_model3]
}

df = pd.DataFrame(data)

# Display the DataFrame
df